# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Import PPrint
from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [56]:
data_file_path = "../WeatherPy/output_data/cities.csv"
city_data = pd.read_csv(data_file_path)

# Drop column that has a duplicate index
city_data = city_data.iloc[:, 1:9]
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Saskylakh,71.9167,114.0833,-19.68,83,6,11.79,RU
1,Korcula,42.9622,17.1369,53.60,94,90,14.97,HR
2,Busselton,-33.6500,115.3333,55.00,94,0,10.18,AU
3,Nikolskoye,59.7035,30.7861,10.99,78,75,2.24,RU
4,Barrow,71.2906,-156.7887,-14.80,76,90,16.11,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [57]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Create base map
fig = gmaps.figure(center = (0, 0), zoom_level = 2)

# Extract location coords and humidity

locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)

# Create a heatmap layer

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 1)

# Add to map

fig.add_layer(heat_layer)

# Display map

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [66]:
# Narrow down cities to fit weather conditions

hotel_df = city_data.loc[(city_data["Max Temp"] >= 70) & 
                         (city_data["Max Temp"] < 80) & 
                         (city_data["Wind Speed"] < 10) & 
                         (city_data["Cloudiness"] == 0)]

# Reset index
hotel_df = hotel_df.reset_index(drop = True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Ushuaia,-54.8000,-68.3000,75.20,38,0,5.75,AR
1,Ancud,-41.8697,-73.8203,77.00,47,0,5.75,CL
2,Jalu,29.0331,21.5482,75.72,16,0,7.20,LY
3,Yarada,17.6500,83.2667,71.60,83,0,2.30,IN
4,Salalah,17.0151,54.0924,77.00,47,0,4.61,OM
5,Tzaneen,-23.8332,30.1635,77.00,47,0,5.75,ZA
6,Visakhapatnam,17.6900,83.2093,71.60,83,0,2.30,IN
7,San Carlos De Bariloche,-41.1456,-71.3082,77.00,38,0,3.44,AR
8,Chuy,-33.6971,-53.4616,76.73,71,0,9.51,UY
9,Victor Harbor,-35.5667,138.6167,78.01,47,0,4.56,AU


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [67]:
# Create an empty column named "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,Ushuaia,-54.8000,-68.3000,75.20,38,0,5.75,AR,
1,Ancud,-41.8697,-73.8203,77.00,47,0,5.75,CL,
2,Jalu,29.0331,21.5482,75.72,16,0,7.20,LY,
3,Yarada,17.6500,83.2667,71.60,83,0,2.30,IN,
4,Salalah,17.0151,54.0924,77.00,47,0,4.61,OM,
5,Tzaneen,-23.8332,30.1635,77.00,47,0,5.75,ZA,
6,Visakhapatnam,17.6900,83.2093,71.60,83,0,2.30,IN,
7,San Carlos De Bariloche,-41.1456,-71.3082,77.00,38,0,3.44,AR,
8,Chuy,-33.6971,-53.4616,76.73,71,0,9.51,UY,
9,Victor Harbor,-35.5667,138.6167,78.01,47,0,4.56,AU,


In [68]:

# Create a params dictionary

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": "",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Retrieve closest hotel to location in coords, using a for loop

for index, row in hotel_df.iterrows():
    
    # Get the city and coords
    city_to_query = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the city to lookup
    params["location"] = f"{lat}, {lng}"
    
    # Assemble URL and make API request
    print(f"Retrieving Results for City #{index}: {city_to_query}")
    response = requests.get(base_url, params = params).json()

    results = response['results']
    
    hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
hotel_df

Retrieving Results for City #0: Ushuaia
Retrieving Results for City #1: Ancud
Retrieving Results for City #2: Jalu
Retrieving Results for City #3: Yarada
Retrieving Results for City #4: Salalah
Retrieving Results for City #5: Tzaneen
Retrieving Results for City #6: Visakhapatnam
Retrieving Results for City #7: San Carlos De Bariloche
Retrieving Results for City #8: Chuy
Retrieving Results for City #9: Victor Harbor
Retrieving Results for City #10: Antofagasta
Retrieving Results for City #11: Asyut
Retrieving Results for City #12: Coihaique
Retrieving Results for City #13: Abnub
Retrieving Results for City #14: Sakakah


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,Ushuaia,-54.8000,-68.3000,75.20,38,0,5.75,AR,Albatros Hotel
1,Ancud,-41.8697,-73.8203,77.00,47,0,5.75,CL,Hotel Arena Gruesa
2,Jalu,29.0331,21.5482,75.72,16,0,7.20,LY,صاحبة الفخامة
3,Yarada,17.6500,83.2667,71.60,83,0,2.30,IN,Jeevan
4,Salalah,17.0151,54.0924,77.00,47,0,4.61,OM,HAMDAN PLAZA HOTEL SALALAH
5,Tzaneen,-23.8332,30.1635,77.00,47,0,5.75,ZA,Hotel @ Tzaneen
6,Visakhapatnam,17.6900,83.2093,71.60,83,0,2.30,IN,Best Western Ramachandra Hotel
7,San Carlos De Bariloche,-41.1456,-71.3082,77.00,38,0,3.44,AR,Hotel Patagonia
8,Chuy,-33.6971,-53.4616,76.73,71,0,9.51,UY,Nuevo Hotel Plaza
9,Victor Harbor,-35.5667,138.6167,78.01,47,0,4.56,AU,McCracken Country Club


In [69]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [70]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(
    locations, 
    info_box_content = hotel_info
)

fig.add_layer(hotel_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))